In [1]:
%pip install faiss-cpu
%pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.1 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached scikit_learn-1.7.1-cp313-cp313-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.1-cp313-cp313-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-non

In [43]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
df = pd.read_csv("sample_text.csv")
df.shape
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [44]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer ("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors.shape

(8, 768)

In [45]:
dim = vectors.shape[1]
dim

768

In [46]:
import faiss
index = faiss.IndexFlatL2()
index 

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x33f17ad60> >

In [47]:
vectors.shape

(8, 768)

In [48]:
import faiss
import numpy as np

# Ensure embeddings are in float32
vectors = np.array(vectors).astype('float32')   # shape (8, 768)

# Dimension must match the second value in shape
d = vectors.shape[1]   # = 768
index = faiss.IndexFlatL2(d)

# Add vectors to FAISS index
index.add(vectors)

print("Number of vectors in index:", index.ntotal)

Number of vectors in index: 8


In [49]:
index.add(vectors)

In [50]:
search_query= "The party was awesome and I had a great time"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [51]:
import numpy as np
svec = np.array(vec).reshape(1, -1)
svec.shape

(1, 768)

In [52]:
# Rebuild clean index
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors.astype('float32'))

In [53]:
distances, I = index.search(svec, k=2)  # k is the number of nearest neighbors to return

In [55]:
df.loc[[4,5]]

,text,category
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
